## · 도커설치
Linux
* curl -s https://get.docker.com/ | sudo sh  
\- 명령어를 입력하고 패스워드를 입력하면 리눅스 배포판에 따라 자동으로 최신버전의 
도커를 설치
* sudo usermod -aG docker ubuntu  
\- ubuntu 유저 권한 추가
* 위처럼 docker 그룹에 사용자를 추가해도 permission denied 발생한다면

    ~$ sudo chown root:docker /var/run/docker.sock: 소켓 파일 그룹 지정

    ~$ sudo chmod 666 /var/run/docker.sock: 소켓 파일 실행권한 변경
    
* docker version  
\- 설치확인

## · 기본 명령어

### 1. run - 컨테이너 실행
\- docker run [OPTIONS] IMAGE[:TAG|@DIGEST] [COMMAND] [ARG...]
* -d: detached mode (백그라운드 모드)
* -p: 호스트와 컨테이너의 포트를 연결
* -v: 호스트와 컨테이너의 디렉토리를 연결
* -e: 컨테이너 내에서 사용할 환경변수 설정
* --name: 컨테이너 이름 설정
* --rm: 프로세스 종료시 컨테이너 자동 제거
* -it: -i와 -t를 동시에 사용한 것으로 터미널 입력을 위한 옵션
* --network: 네트워크 연결

**ex) ubuntu 20.04 컨테이너 만들기**
- docker run ubuntu:20.04

=> 사용할 이미지가 저장되어 있는지 확인하고 없다면 다운로드(pull) 한 후 컨테이너를 생성(create)하고 시작(start)한다.

**ex) /bin/sh 실행하기**
- docker run --rm -it ubuntu:20.04 /bin/sh

=> 쉘 사용을 위해 sh를 실행하고 키보드 입력을 위해 -it 옵션

**ex) 웹 어플리케이션 실행하기**
- docker run --rm -p 5678:5678 hashicorp/http-echo -text="hello world"

=> localhost:5678에 접속하면 메시지를 볼 수 있다.

**ex) MySQL 실행하기**
1. mysql 컨테이너 생성
- docker run -d -p 3306:3306 \
    -e MYSQL_ALLOW_EMPTY_PASSWORD=true \
    --name mysql \
    mysql:5.7
    
2. mysql 데이터베이스 실행
- docker exec -it mysql mysql  
    create database wp CHARACTER SET utf8;  
    grant all privileges on wp.* to wp@'%' identified by 'wp';  
    flush privileges;  
    quit  

**ex) 워드프레스 블로그 실행하기**
- docker run -d -p 8080:80 \
    -e WORDPRESS_DB_HOST=host.docker.internal \
    -e WORDPRESS_DB_NAME=wp \
    -e WORDPRESS_DB_USER=wp \
    -e WORDPRESS_DB_PASSWORD=wp \
    wordpress

=> 앞에서 만든 MySQL을 실행한 상태에서 생생, 웹브라우저 localhost:8080으로 접속

### 2. exec
\- exec 명령어는 run 명령어와 달리 실행중인 도커 컨테이너에 접속할 때 사용하며 컨테이너 안에 ssh server등을 설치하지 않고 exec 명령어로 접속한다.

### 3. ps
- docker ps: 현재 실행중인 컨테이너 목록
- docker ps -a: 중지된 컨테이너 목록까지 확인

### 4. stop
\- docker stop [OPTIONS] CONTAINER [CONTAINER...]  
: 실행중인 컨테이너를 중지하는 명령어.  
실행중인 컨테이너를 하나 또는 여러개 (띄어쓰기) 중지할 수 있다.

### 5. logs
- docker logs [OPTIONS] CONTAINER  
: 컨테이너가 정상적으로 동작하는지 확인  
=> 기본 옵션과 -f, --tail 참고

## 6. images
\- docker images
: 도커가 다운로드한 이미지 목록 확인

## 7. pull
\- docker pull ubuntu:18.04
: 이미지를 다운로드하는 명령어

### 8. rmi
\- docker rmi [OPTIONS] IMAGE [IMAGE...]  
: 다운로드 된 이미지 삭제  
=> 단, 컨테이너가 실행중인 이미지는 삭제되지 않는다.


### 9. network create
\- docker network create [OPTIONS] NETWORK  
=> 도커 컨테이너끼리 이름으로 통신할 수 있는 가상 네트워크를 만든다.

ex) docker network create app-network

### 10. network connect
\- docker network connect [OPTIONS] NETWORK CONTAINER  
=> 기존에 생성된 컨테이너에 네트워크를 추가한다.

ex) docker network connect app-network mysql

## 11. volume mount (-v)
* mysql과 wordpress를 연결해서 사용하다가 mysql 컨테이너를 종료하고 삭제한 후에 다시 wordpress를 접속하면 db error가 뜬다.  
왜냐 mysql 컨테이너가 삭제되면서 데이터도 함께 사라지기 때문이다.  
이를 방지하기 위해 host 디렉토리와 컨테이너 디렉토리를 연결하여 데이터를 저장해두기 위해 -v 옵션 사용

ex)  
    1. mysql과 wordpress 연결
    2. mysql 삭제 후 다시 생성
    3. wordpress 실행 -> db error
    4. mysql 다시 삭제 후 -v 옵션을 추가하여 다시 mysql 생성
    5. exec mysql로 db 다시 생성
    6. 다시 mysql 삭제 후 -v 옵션으로 생성해보면 wordpress와 성공적으로 연결

* docker stop mysql  
  docker rm mysql  
    docker run -d -p 3306:3306 \
    -e MYSQL_ALLOW_EMPTY_PASSWORD=true \
    --network=app-network \
    --name mysql \
    -v '연결하려는 호스트 디렉토리':/var/lib/mysql \
    mysql:5.7